### FINA 4380 with Marius Popescu

### Portfolio Risk and Return - Exercise Set Solutions

#### Import the Necessary Libraries

In [2]:
import numpy as np
import pandas as pd

import datetime as dt
import yfinance as yf
import pandas_datareader.data as web

#### 1. Create a DataFrame named data3 to store the Adjusted Closing Price from Yahoo!Finance for Goldman Sachs (ticker= 'GS'), Coca-Cola (ticker = 'KO'), and IBM (ticker= 'IBM') from January 1st, 2015 to December 31st, 2022.

In [20]:
start = dt.datetime(2015, 1, 1)-pd.offsets.BDay(1)
end = dt.datetime(2022, 12, 31)+pd.offsets.BDay(1)

ticker_list= ['GS','KO','IBM']

data3 = yf.download(ticker_list, start, end)['Adj Close']
data3.columns=[ticker+'_PRC'for ticker in ticker_list]
data3.head()

[*********************100%***********************]  3 of 3 completed


,GS_PRC,KO_PRC,IBM_PRC
Date,,,
2014-12-31,166.912033,107.067566,32.408779
2015-01-02,167.411438,108.148643,32.347389
2015-01-05,162.184448,106.446938,32.347389
2015-01-06,158.903549,104.151299,32.593006
2015-01-07,161.271652,103.470589,32.999855


#### 2. Create three additional columns in DataFrame data3 to store the daily simple returns for the three stocks.

In [21]:
for ticker in ticker_list:
    data3[ticker+'_RET'] = data3[ticker+'_PRC'].pct_change()
data3.head()

,GS_PRC,KO_PRC,IBM_PRC,GS_RET,KO_RET,IBM_RET
Date,,,,,,
2014-12-31,166.912033,107.067566,32.408779,NaN,NaN,NaN
2015-01-02,167.411438,108.148643,32.347389,0.002992,0.010097,-0.001894
2015-01-05,162.184448,106.446938,32.347389,-0.031222,-0.015735,0.000000
2015-01-06,158.903549,104.151299,32.593006,-0.020229,-0.021566,0.007593
2015-01-07,161.271652,103.470589,32.999855,0.014903,-0.006536,0.012483


#### 3. Use the below provided random seed to generate an array of portfolio weights from the U(0,1) distribution.

In [22]:
np.random.seed(500)
weights = np.array(np.random.uniform(0,1,3))
weights = weights / np.sum(weights)

print(weights)

[0.48781685 0.04340129 0.46878185]


#### 4. Create a new DataFrame labeled portfolios to store the return of a daily-rebalanced portfolio that invests in the three stocks according to the weights from question 3. Label the column 'DR_port_ret'.

In [23]:
portfolios = pd.DataFrame(index=data3.index)
portfolios['DR_port_ret'] = np.dot(data3[['GS_RET','KO_RET','IBM_RET']], weights)
#portfolios['DR_port_ret'] = np.dot(data3[[tick+'_RET' for tick in ticker_list]], weights)

In [24]:
portfolios.head()

,DR_port_ret
Date,
2014-12-31,NaN
2015-01-02,0.001010
2015-01-05,-0.015914
2015-01-06,-0.007245
2015-01-07,0.012838


#### 5. Create a new column in the DataFrame portfolios to store the value of a $1 investment in the daily-rebalanced portfolio from question 4. Label the column 'DR_port_val'

In [27]:
portfolios['DR_port_val'] = portfolios[['DR_port_ret']].apply(lambda x:x.add(1).cumprod())
portfolios['DR_port_val'].iloc[0]=1

In [28]:
portfolios.head()

,DR_port_ret,DR_port_val
Date,,
2014-12-31,NaN,1.000000
2015-01-02,0.001010,1.001010
2015-01-05,-0.015914,0.985080
2015-01-06,-0.007245,0.977943
2015-01-07,0.012838,0.990498


#### 6. Create a new column in the DataFrame porfolios to store the value of a Buy and Hold strategy that invests $1 in three stocks according to the weights in question 3. Label the column 'BH_port_val'.

In [29]:
portfolios['BH_port_val'] = np.dot(data3[['GS_PRC','KO_PRC','IBM_PRC']]/data3[['GS_PRC','KO_PRC','IBM_PRC']].iloc[0], weights)
#portfolios['BH_Port_val'] = np.dot(data3[[tick+'_PRC' for tick in ticker_list]]/data3[[tick+'_PRC' for tick in ticker_list]].iloc[0], weights)

In [30]:
portfolios.head()

,DR_port_ret,DR_port_val,BH_port_val
Date,,,
2014-12-31,NaN,1.000000,1.000000
2015-01-02,0.001010,1.001010,1.001010
2015-01-05,-0.015914,0.985080,0.985044
2015-01-06,-0.007245,0.977943,0.978077
2015-01-07,0.012838,0.990498,0.990607


#### 7. Create a new column in the DataFrame porfolios to store the return of the Buy and Hold strategy from question 6. Label the column 'BH_port_ret'.

In [31]:
portfolios['BH_port_ret'] = portfolios['BH_port_val'].pct_change()

In [32]:
portfolios.head()

,DR_port_ret,DR_port_val,BH_port_val,BH_port_ret
Date,,,,
2014-12-31,NaN,1.000000,1.000000,NaN
2015-01-02,0.001010,1.001010,1.001010,0.001010
2015-01-05,-0.015914,0.985080,0.985044,-0.015950
2015-01-06,-0.007245,0.977943,0.978077,-0.007072
2015-01-07,0.012838,0.990498,0.990607,0.012811


#### 8. Compute each portfolio's annualized risk?

In [44]:
port_risk = portfolios[['DR_port_ret','BH_port_ret']].dropna().std()* np.sqrt(250)
port_risk

DR_port_ret    0.205954
BH_port_ret    0.203383
dtype: float64

#### 9. Compute the monthly returns for the two portfolios and store them in a new DataFrame labeled portfolios_mon.

In [45]:
portfolios_mon = portfolios[['DR_port_ret','BH_port_ret']].dropna().resample(rule='M').apply(lambda x: x.add(1).prod().sub(1))
portfolios_mon.head()

,DR_port_ret,BH_port_ret
Date,,
2015-01-31,-0.067957,-0.067495
2015-02-28,0.078072,0.076710
2015-03-31,-0.031160,-0.031658
2015-04-30,0.025022,0.025081
2015-05-31,0.030270,0.030774


#### 10. Compute each portfolio's Sharpe ratio

In [39]:
rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']]/100
rf.index = rf.index.to_timestamp(how='end').normalize()
rf.head()

,RF
Date,
2014-12-31,0.0
2015-01-31,0.0
2015-02-28,0.0
2015-03-31,0.0
2015-04-30,0.0


In [40]:
data = pd.merge(portfolios_mon, rf, on = 'Date')
data['DR_ex_ret'] = data['DR_port_ret'] - data['RF']
data['BH_ex_ret'] = data['BH_port_ret'] - data['RF']
data.head()

,DR_port_ret,BH_port_ret,RF,DR_ex_ret,BH_ex_ret
Date,,,,,
2015-01-31,-0.067957,-0.067495,0.0,-0.067957,-0.067495
2015-02-28,0.078072,0.076710,0.0,0.078072,0.076710
2015-03-31,-0.031160,-0.031658,0.0,-0.031160,-0.031658
2015-04-30,0.025022,0.025081,0.0,0.025022,0.025081
2015-05-31,0.030270,0.030774,0.0,0.030270,0.030774


In [41]:
SR_port = (data[['DR_ex_ret','BH_ex_ret']].mean()*12)/(data[['DR_ex_ret','BH_ex_ret']].std()*np.sqrt(12))
SR_port

DR_ex_ret    0.539087
BH_ex_ret    0.494989
dtype: float64

In [42]:
SR_port = (data[[i+'_ex_ret' for i in(['DR','BH'])]].mean()*12)/(data[[i+'_ex_ret' for i in(['DR','BH'])]].std()*np.sqrt(12))
SR_port

DR_ex_ret    0.539087
BH_ex_ret    0.494989
dtype: float64